In [ ]:
#---------------------------------------------------------------------- GeoWatch Tower ----------------------------------------------------------------------------------------#


In [ ]:
#--------------------------------- PIP INSTALLS ----------------------------------
!pip install neo4j
!pip install geopy
!pip install requests
!pip install scikit-learn
!pip install pandas
!pip install networkx


In [ ]:
# --------------------------- config.py ----------------------------------
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "your_password"

IP_LOOKUP_API = "https://api.ipgeolocation.io/ipgeo"
IP_LOOKUP_API_KEY = "your_api_key"

HONEYPOT_SECRET_URLS = ["/free-gift", "/exclusive-offer", "/click-here-now"]
SUSPICIOUS_SPEED_THRESHOLD_KMH = 1000


In [ ]:
# -------------------------------------- geo_enrichment.py -----------------------------------
import requests
from config import IP_LOOKUP_API, IP_LOOKUP_API_KEY

def enrich_ip(ip_address):
    params = {
        'apiKey': IP_LOOKUP_API_KEY,
        'ip': ip_address
    }
    try:
        response = requests.get(IP_LOOKUP_API, params=params)
        data = response.json()
        return {
            'ip': ip_address,
            'latitude': float(data.get('latitude', 0)),
            'longitude': float(data.get('longitude', 0)),
            'country': data.get('country_name', ''),
            'timezone': data.get('time_zone', {}).get('name', '')
        }
    except Exception as e:
        print(f"Geo enrichment failed for {ip_address}: {e}")
        return None


In [ ]:
# -------------------------------------- behavior_analysis.py ------------------------------------
from geopy.distance import geodesic
from config import SUSPICIOUS_SPEED_THRESHOLD_KMH

def calculate_speed_kmh(prev_location, curr_location, time_diff_hours):
    distance_km = geodesic(
        (prev_location['latitude'], prev_location['longitude']),
        (curr_location['latitude'], curr_location['longitude'])
    ).km
    if time_diff_hours == 0:
        return float('inf')
    return distance_km / time_diff_hours

def detect_impossible_travel(user_sessions):
    for i in range(1, len(user_sessions)):
        prev = user_sessions[i-1]
        curr = user_sessions[i]
        time_diff = (curr['timestamp'] - prev['timestamp']).total_seconds() / 3600.0
        speed = calculate_speed_kmh(prev['location'], curr['location'], time_diff)
        if speed > SUSPICIOUS_SPEED_THRESHOLD_KMH:
            return True
    return False


In [ ]:
# --------------------------------------------- graph_analysis.py ----------------------------------------
from neo4j import GraphDatabase
from config import NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD

class GraphManager:
    def __init__(self):
        self.driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

    def close(self):
        self.driver.close()

    def create_user_node(self, user_id, location):
        with self.driver.session() as session:
            session.run(
                "MERGE (u:User {id: $user_id}) SET u.latitude = $latitude, u.longitude = $longitude",
                user_id=user_id,
                latitude=location['latitude'],
                longitude=location['longitude']
            )

    def create_interaction(self, user_id_1, user_id_2):
        with self.driver.session() as session:
            session.run(
                "MATCH (a:User {id: $user_id_1}), (b:User {id: $user_id_2}) MERGE (a)-[:INTERACTED_WITH]->(b)",
                user_id_1=user_id_1,
                user_id_2=user_id_2
            )

    def detect_clusters(self):
        with self.driver.session() as session:
            result = session.run(
                "CALL gds.graph.project('userGraph', 'User', 'INTERACTED_WITH') YIELD graphName "
                "CALL gds.louvain.stream('userGraph') YIELD nodeId, communityId "
                "RETURN communityId, count(nodeId) as size ORDER BY size DESC"
            )
            return [(record['communityId'], record['size']) for record in result]


In [ ]:
# -------------------------------------- anomaly_detection.py -------------------------------
from sklearn.ensemble import IsolationForest

class AnomalyDetector:
    def __init__(self):
        self.model = IsolationForest(contamination=0.02)

    def train(self, feature_vectors):
        self.model.fit(feature_vectors)

    def predict(self, feature_vectors):
        return self.model.predict(feature_vectors)

# -------------------------------------- honeypot.py --------------------------------------
from config import HONEYPOT_SECRET_URLS

def is_honeypot_triggered(request_path):
    return request_path in HONEYPOT_SECRET_URLS



In [ ]:
# ----------------------------------------- main.py --------------------------------------------
from geo_enrichment import enrich_ip
from behavior_analysis import detect_impossible_travel
from graph_analysis import GraphManager
from honeypot import is_honeypot_triggered
from anomaly_detection import AnomalyDetector
from datetime import datetime, timedelta

def simulate_geo_watch():
    user_sessions = [
        {'timestamp': datetime.utcnow(), 'location': enrich_ip('8.8.8.8')},
        {'timestamp': datetime.utcnow() + timedelta(hours=1), 'location': enrich_ip('5.5.5.5')}
    ]

    if detect_impossible_travel(user_sessions):
        print("[ALERT] Impossible travel detected!")

    graph = GraphManager()
    graph.create_user_node("user123", user_sessions[0]['location'])
    graph.create_user_node("user456", user_sessions[1]['location'])
    graph.create_interaction("user123", "user456")
    clusters = graph.detect_clusters()
    print(f"Detected Clusters: {clusters}")
    graph.close()

    if is_honeypot_triggered("/free-gift"):
        print("[ALERT] Honeypot triggered!")

    features = [[0.5, 1.2], [0.6, 1.1], [50.5, 60.2]]
    detector = AnomalyDetector()
    detector.train(features)
    predictions = detector.predict(features)
    print(f"Anomaly Predictions: {predictions}")

if __name__ == "__main__":
    simulate_geo_watch()
